# Memory

The Memory is a list of bytes. Each byte can be accessed individually.

Similar to RAM, it starts of by being completely empty. It is also volatile, which means that after execution the memory is reset.

Each memory cell can hold 1 byte (8 bits). A number between 0 and 255.

![title](../static/memory.png)

We will represent Memory as a simple list that can be accessed by an index or `offset`. 

An offset of `2` would mean that we get the byte stored at index `2`. Combined with a `size` we can get a block of bytes. Offset `2` and size `5` would return the bytes from index `2` to index `7`.

In [ ]:
struct Memory {
    memory: Vec<u8>,
}

impl Memory {
    fn new() -> Self {
        Memory { memory: vec![] }
    }

    fn access(&self, offset: usize, size: usize) -> &[u8] {
        &self.memory[offset..offset + size]
    }

    fn load(&self, offset: usize) -> &[u8] {
        self.access(offset, 32)
    }

    fn store(&mut self, offset: usize, value: &[u8]) {
        let end = offset + value.len();

        if end > self.memory.len() {
            self.memory.resize(end, 0);
        }

        self.memory[offset..end].copy_from_slice(value);
    }
}

Expanding the size of Memory consumes gas non-linearly. Making it more costly to create larger and larger Memory space.

Lets add the gas expansion calculation to the Memory

In [ ]:
struct Memory {
    memory: Vec<u8>,
}

impl Memory {
    fn new() -> Self {
        Memory { memory: vec![] }
    }

    fn store(&mut self, offset: usize, value: &[u8]) -> usize {
        let mut memory_expansion_cost = 0;

        if self.memory.len() <= offset + value.len() {
            let mut expansion_size = 0;

            // initialize memory with 32 zeros if it is empty
            if self.memory.len() == 0 {
                expansion_size = 32;
                self.memory = vec![0x00; 32];
            }

            // extend memory more if needed
            if self.memory.len() < offset + value.len() {
                expansion_size += offset + value.len() - self.memory.len();
                self.memory.extend(std::iter::repeat(0x00).take(expansion_size));
            }

            memory_expansion_cost = expansion_size * expansion_size; // simplified!
        }

        // Call parent store behavior (same as SimpleMemory)
        let end = offset + value.len();
        self.memory[offset..end].copy_from_slice(value);

        memory_expansion_cost
    }
}


Lets create a some Memory

In [ ]:
let mut memory = Memory::new();

We store 4 values at offset 0

In [ ]:
let cost = memory.store(0, &[0x01, 0x02, 0x03, 0x04]);

`load` will always return 32 bytes if it can

In [ ]:
let data = memory.load(0);

[1,
 2,
 3,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

### Advanced

This is the actual way how the gas cost for a memory expansion is calculated.

In [ ]:
fn calc_memory_expansion_gas(memory_byte_size: usize) -> usize {
    let memory_size_word = (memory_byte_size + 31) / 32;
    let memory_cost = (memory_size_word * memory_size_word) / 512 + (3 * memory_size_word);
    memory_cost
}